In [1]:
from model4LHzc import *
from data_3ch import *

vid="v049_5" #version id should match the file number 
# The final number can be [1:5] and denote the cross-validation order
# Ran DellWS with GeForce RTX3060 GPU

# Modified the model4LHzc input dimension from 512x512x1 to 512x512x3

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=45,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

# SETTINGS ***
batch_size=3
learning_rate=1e-3

train_gen = trainGenerator(batch_size,'afm_cell_512/train0'+vid[-1],'afmheideffri','labels',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'afm_cell_512/valid0'+vid[-1],'afmheideffri','labels',data_gen_args,save_to_dir = None)

# 24 images are used for training, 6 images for validating and 4 images for testing
train_steps = 24//batch_size
valid_steps = 6//batch_size

# SETTINGS ***
loss=dice_loss
steps_per_epoch=20*train_steps
num_epochs=100

model = unet(learning_rate, loss)

opt = tf.keras.optimizers.Adam(learning_rate)
metrics = ["acc", dice_coef, iou]
model.compile(loss=dice_loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files/unet_cell_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps, 
                    epochs=num_epochs, callbacks=callbacks)


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 24 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Epoch 1/100
160/160 [==============================] - ETA: 0s - loss: 0.2512 - acc: 0.8868 - dice_coef: 0.7488 - iou: 0.6180Found 6 images belonging to 1 classes.
Found 6 images belonging to 1 classes.
160/160 [==============================] - 91s 484ms/step - loss: 0.2512 - acc: 0.8868 - dice_coef: 0.7488 - iou: 0.6180 - val_loss: 0.9604 - val_acc: 0.7344 - val_dice_coef: 0.0396 - val_iou: 0.0202

Epoch 00001: saving model to files\unet_cell_seg_v049_5.hdf5
Epoch 2/100
160/160 [==============================] - 76s 479ms/step - loss: 0.1123 - acc: 0.9523 - dice_coef: 0.8877 - iou: 0.8034 - val_loss: 0.7097 - val_acc: 0.7650 - val_dice_coef: 0.2903 - val_iou: 0.1746

Epoch 00002: saving model to files\unet_cell_seg_v049_5.hdf5
Epoch 3/100
160/160 [==============================] - 73s 456ms/step - loss: 0.0830 - acc: 0.9627 - dice_coef: 0.9170 - iou: 0.8504 - val_loss: 0.3949 - val_acc: 0.8302 - val_dice_

### validate your model and save predicted results

In [3]:
validGene = testGenerator("afm_cell_512/valid0"+vid[-1]+"/predhdf")
model.load_weights("files/unet_cell_seg_"+vid+".hdf5")
results = model.predict_generator(validGene,6,verbose=1)
saveResult("afm_cell_512/valid0"+vid[-1]+"/predhdf",results,vid[:4])

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


6/6 [==============================] - 2s 106ms/step
